
# Database Table Structures

## Users Table
| Column Name | Data Type     | Constraints          |
|-------------|---------------|----------------------|
| id          | INT           | PRIMARY KEY          |
| name        | VARCHAR(50)   | NOT NULL             |
| email       | VARCHAR(100)  | UNIQUE, NOT NULL     |

---

## Posts Table
| Column Name | Data Type      | Constraints            |
|-------------|----------------|------------------------|
| id          | INT            | PRIMARY KEY            |
| title       | VARCHAR(255)   | NOT NULL               |
| content     | TEXT           |                        |
| user_id     | INT            | FOREIGN KEY (users.id) |

---

## Tags Table
| Column Name | Data Type     | Constraints        |
|-------------|---------------|--------------------|
| id          | INT           | PRIMARY KEY        |
| name        | VARCHAR(50)   | UNIQUE, NOT NULL   |

---

## Post_Tag Table
| Column Name | Data Type | Constraints            |
|-------------|-----------|------------------------|
| post_id     | INT       | PRIMARY KEY, FOREIGN KEY (posts.id) |
| tag_id      | INT       | PRIMARY KEY, FOREIGN KEY (tags.id)  |

```sql
CREATE TABLE users (
    id INT PRIMARY KEY,
    name VARCHAR(50) NOT NULL,
    email VARCHAR(100) UNIQUE NOT NULL
);

-- Assuming you have a posts table with a user_id foreign key
CREATE TABLE posts (
    id INT PRIMARY KEY,
    title VARCHAR(255) NOT NULL,
    content TEXT,
    user_id INT,
    FOREIGN KEY (user_id) REFERENCES users(id)
);

CREATE TABLE posts (
    id INT PRIMARY KEY,
    title VARCHAR(100) NOT NULL,
    content VARCHAR(100) NOT NULL,
    user_id INT NOT NULL,
    FOREIGN KEY (user_id) REFERENCES users(id)
);

-- Association table for the Many-to-Many relationship
CREATE TABLE post_tag (
    post_id INT,
    tag_id INT,
    PRIMARY KEY (post_id, tag_id),
    FOREIGN KEY (post_id) REFERENCES posts(id),
    FOREIGN KEY (tag_id) REFERENCES tags(id)
);
CREATE TABLE tags (
    id INT PRIMARY KEY,
    name VARCHAR(50) UNIQUE NOT NULL
);

```

In [ ]:
# Install required libraries (Run this if not already installed)
!pip install sqlalchemy pymysql

In [ ]:

from sqlalchemy import create_engine
from sqlalchemy.orm import declarative_base, sessionmaker

# Replace these credentials with your MySQL details
DATABASE_URL = "mysql+pymysql://<username>:<password>@<host>/<database>"

# Create the engine
engine = create_engine(DATABASE_URL, echo=True)

# Define the Base class for the ORM models
Base = declarative_base()

# Create a session maker
Session = sessionmaker(bind=engine)
session = Session()
    

In [ ]:
from sqlalchemy import  MetaData, Table, Column, Integer, String, Text, ForeignKey

# Create MetaData object
metadata = MetaData()

# Define the 'users' table
users_table = Table(
    "users", metadata,
    Column("id", Integer, primary_key=True),
    Column("name", String(50), nullable=False),
    Column("email", String(100), unique=True, nullable=False)
)

# Define the 'posts' table
posts_table = Table(
    "posts", metadata,
    Column("id", Integer, primary_key=True),
    Column("title", String(255), nullable=False),
    Column("content", Text),
    Column("user_id", Integer, ForeignKey("users.id"))
)

# Create all tables in the database
metadata.create_all(engine)

display("Tables created successfully!")


In [ ]:
# Perform raw SQL queries
with engine.connect() as conn:
    # Insert raw data
    conn.execute("INSERT INTO users (id, name, email) VALUES (1, 'Alice', 'alice@example.com')")
    conn.execute("INSERT INTO users (id, name, email) VALUES (2, 'Bob', 'bob@example.com')")

    # Retrieve raw data
    result = conn.execute("SELECT * FROM users")
    display("Raw Query Results:")
    for row in result:
        display(row)

    # Update data
    conn.execute("UPDATE users SET name = 'Alice Smith' WHERE id = 1")

    # Delete data
    conn.execute("DELETE FROM users WHERE id = 2")

with engine.begin() as conn:  # Ensures transactional safety
    conn.execute("DELETE FROM users")
    display("All data deleted transactionally!")

In [ ]:
# Perform Tablural SQL queries

with engine.connect() as conn:
    # Insert data into 'users' table
    conn.execute(users_table.insert().values(name="Alice", email="alice@example.com"))
    conn.execute(users_table.insert().values(name="Bob", email="bob@example.com"))

    # Insert data into 'posts' table
    conn.execute(posts_table.insert().values(title="First Post", content="Hello World!", user_id=1))
    conn.execute(posts_table.insert().values(title="Second Post", content="SQLAlchemy is cool!", user_id=2))

    display("Data inserted successfully!")


In [ ]:
with engine.connect() as conn:
    # Retrieve all users
    result = conn.execute(users_table.select())
    
    display("Users:")
    
    for row in result:
        display(row)

    # Retrieve all posts
    result = conn.execute(posts_table.select())
    
    display("\nPosts:")
    
    for row in result:
        display(row)


In [ ]:
with engine.connect() as conn:
    # Update a user's email
    conn.execute(users_table.update().where(users_table.c.name == "Alice").values(email="alice_new@example.com"))
    display("User updated successfully!")


In [ ]:
with engine.connect() as conn:
    # Delete a user
    conn.execute(users_table.delete().where(users_table.c.name == "Bob"))
    display("User deleted successfully!")


In [ ]:

# Drop the specific table
users_table.drop(engine)
display("Users table dropped successfully!")

# Drop all tables in the metadata
metadata.drop_all(engine)
display("All tables dropped successfully!")

In [ ]:
# Define the 'users' table
users_table = Table(
    "users", metadata,
    Column("id", Integer, primary_key=True),
    Column("name", String(50), nullable=False),
    Column("email", String(100), unique=True, nullable=False)
)

# Define the 'posts' table
posts_table = Table(
    "posts", metadata,
    Column("id", Integer, primary_key=True),
    Column("title", String(255), nullable=False),
    Column("content", Text),
    Column("user_id", Integer, ForeignKey("users.id"))
)

# Create all tables in the database
metadata.create_all(engine)

display("Tables created successfully!")


In [ ]:
# Perform raw SQL queries
# Insert raw data
session.execute("INSERT INTO users (id, name, email) VALUES (1, 'Alice', 'alice@example.com')")
session.execute("INSERT INTO users (id, name, email) VALUES (2, 'Bob', 'bob@example.com')")
session.commit()

# Retrieve raw data
result = session.execute("SELECT * FROM users")
display("Raw Query Results:")
for row in result:
    display(row)

# Update raw data
session.execute("UPDATE users SET name = 'Alice Smith' WHERE id = 1")
session.commit()

# Delete raw data
session.execute("DELETE FROM users WHERE id = 2")
session.commit()


In [ ]:

# Drop the specific table 
with engine.begin() as conn:
    conn.execute("DROP TABLE IF EXISTS users")
 

# Drop all tables in the metadata
with engine.connect() as conn:
    # Retrieve all table names from the database
    result = conn.execute("SELECT name FROM sqlite_master WHERE type='table'")
    tables = [row[0] for row in result]

    # Drop each table
    for table in tables:
        conn.execute(f"DROP TABLE IF EXISTS {table}")
        display(f"Table '{table}' dropped successfully!")

    display("All tables dropped successfully!")


In [ ]:
from sqlalchemy import create_engine, MetaData, Table, Column, Integer, String, ForeignKey
from sqlalchemy.sql import select
 

# Define the Users table
users_table = Table(
    'users', metadata,
    Column('id', Integer, primary_key=True),
    Column('name', String(50), nullable=False),
    Column('email', String(100), unique=True, nullable=False)
)

# Define the Posts table
posts_table = Table(
    'posts', metadata,
    Column('id', Integer, primary_key=True),
    Column('title', String(100), nullable=False),
    Column('content', String, nullable=False),
    Column('user_id', Integer, ForeignKey('users.id'), nullable=False)
)

# Define the Tags table
tags_table = Table(
    'tags', metadata,
    Column('id', Integer, primary_key=True),
    Column('name', String(50), unique=True, nullable=False)
)

# Define the Association table for Many-to-Many relationship
post_tag_table = Table(
    'post_tag', metadata,
    Column('post_id', Integer, ForeignKey('posts.id'), primary_key=True),
    Column('tag_id', Integer, ForeignKey('tags.id'), primary_key=True)
)

# Create all tables in the database
metadata.create_all(engine)

# Example usage (insert and query)
with engine.connect() as connection:
    # Insert data into the Users table
    insert_user = users_table.insert().values(name="Alice", email="alice@example.com")
    connection.execute(insert_user)
    
    # Query data from the Users table
    select_query = select([users_table])
    result = connection.execute(select_query)
    for row in result:
        display(row)

    # Insert data into Posts and Tags
    insert_post = posts_table.insert().values(title="First Post", content="This is a post.", user_id=1)
    insert_tag = tags_table.insert().values(name="Technology")
    connection.execute(insert_post)
    connection.execute(insert_tag)

    # Add a relationship between a post and a tag
    insert_post_tag = post_tag_table.insert().values(post_id=1, tag_id=1)
    connection.execute(insert_post_tag)

    # Query to join posts with tags
    join_query = select([posts_table.c.title, tags_table.c.name]).select_from(
        posts_table.join(post_tag_table).join(tags_table)
    )
    result = connection.execute(join_query)
    for row in result:
        display(row)
